# Fuite de données
Trouvez et résolvez ce problème qui ruine votre modèle de manière subtile.


## Concepts clés et résumé
La fuite de données est un problème critique dans l'apprentissage automatique qui peut ruiner la précision de vos modèles en production. Il existe deux types principaux de fuite : *la fuite de cible* et *la contamination train-test*.

----

### 1. Fuite de cible (Target Leakage)
- Définition : Les prédicteurs incluent des informations qui ne seraient pas disponibles lors de l'utilisation du modèle en production.
- Exemple : Dans une prédiction de pneumonie, inclure une variable comme **took_antibiotic_medicine** qui est modifiée après l'apparition de la maladie crée une fuite.
- Conséquence : Le modèle performe bien sur l'entraînement et la validation, mais échoue en production.
- Solution : Exclure les variables mises à jour après la détermination de la cible.

### 2. Contamination train-test
- Définition : La séparation entre les données d'entraînement et de validation est mal gérée.
- Exemple : Effectuer un prétraitement comme l'imputation de valeurs manquantes avant le partage train-test peut introduire des informations des données de validation dans l'entraînement.
- Conséquence : Des scores de validation optimistes, mais une mauvaise généralisation sur de nouvelles données.
- Solution : Effectuer tous les prétraitements dans des pipelines scikit-learn pour garantir une séparation stricte des ensembles.

---

### **Étude de cas : Dataset des demandes de carte de crédit**

#### **Étapes principales :**
1. **Problème détecté :**
   - Variables suspectes : `expenditure` et `share`, qui semblent être liées à des dépenses après approbation de la carte.
   - Analyse simple des données :
     ```python
     expenditures_cardholders = X.expenditure[y]
     expenditures_noncardholders = X.expenditure[~y]
     
     print('Non-cardholders sans dépenses: %.2f' \
           %((expenditures_noncardholders == 0).mean()))
     print('Cardholders sans dépenses: %.2f' \
           %((expenditures_cardholders == 0).mean()))
     ```
   - Résultat : Tous les non-cardholders avaient des dépenses nulles, tandis que seuls 2 % des cardholders avaient des dépenses nulles.

2. **Correction :**
   - Suppression des variables suspectes (`expenditure`, `share`, `active`, `majorcards`).
   - Réévaluation du modèle :
     ```python
     potential_leaks = ['expenditure', 'share', 'active', 'majorcards']
     X2 = X.drop(potential_leaks, axis=1)
     
     cv_scores = cross_val_score(my_pipeline, X2, y, cv=5, scoring='accuracy')
     print("Cross-val accuracy: %f" % cv_scores.mean())
     ```
   - Nouveau score : **0.830919**. Plus faible, mais fiable en production.

---

### **Conclusion**
- **Impact :** Les modèles avec fuite de données semblent performants en validation, mais échouent sur de nouvelles données.
- **Prévention :**
  - Éviter les variables créées après la détermination de la cible.
  - Utiliser des pipelines pour garantir une séparation stricte entre entraînement et validation.

### **Prochaines étapes**
- S'entraîner à détecter les fuites dans des exemples concrets.
- Utiliser des outils comme **scikit-learn pipelines** pour sécuriser les processus de prétraitement.

Si vous souhaitez des exercices pratiques ou des exemples supplémentaires pour approfondir ce concept, je peux vous aider à les mettre en place !

---

Créons un exemple pratique pour identifier et corriger les fuites de données. Nous utiliserons un jeu de données fictif sur des prêts bancaires pour prédire si un client remboursera son prêt. Nous simulerons une fuite de données pour voir son effet sur les résultats, puis nous corrigerons le problème.

### Exemple : Prévision de remboursement de prêts

#### **1. Préparation des données**
Commençons par créer des données fictives avec une fuite volontaire.

```python
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline

# Création des données fictives
np.random.seed(42)
data = pd.DataFrame({
    'age': np.random.randint(18, 70, size=500),
    'income': np.random.uniform(20000, 80000, size=500),
    'loan_amount': np.random.uniform(5000, 40000, size=500),
    'repayment_status': np.random.choice([0, 1], size=500, p=[0.3, 0.7]),
    # Variable introduisant une fuite (indique si le prêt a été remboursé)
    'repayment_history_score': np.random.choice([0.2, 0.8], size=500, p=[0.3, 0.7])
})

# La variable "repayment_history_score" reflète directement la cible
data['repayment_history_score'] = data['repayment_status'] * 0.8 + (1 - data['repayment_status']) * 0.2

# Cible et prédicteurs
X = data.drop('repayment_status', axis=1)
y = data['repayment_status']

# Séparation des données
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

# Affichage des données
print(data.head())
```

#### **2. Modélisation avec fuite**
Testons un modèle en incluant la variable "repayment_history_score", qui introduit une fuite.

```python
# Modèle avec fuite
pipeline = make_pipeline(RandomForestClassifier(n_estimators=100, random_state=42))
pipeline.fit(X_train, y_train)

# Évaluation
accuracy_with_leak = pipeline.score(X_valid, y_valid)
print(f"Accuracy avec fuite : {accuracy_with_leak:.2f}")
```

#### **3. Identification de la fuite**
Analysez la corrélation entre les prédicteurs et la cible pour identifier des relations suspectes.

```python
# Analyse des corrélations
correlations = data.corr()
print(correlations['repayment_status'].sort_values(ascending=False))
```

#### **4. Suppression de la fuite**
Excluons la variable "repayment_history_score" et réentraînons le modèle.

```python
# Suppression de la variable avec fuite
X_train_no_leak = X_train.drop(columns=['repayment_history_score'])
X_valid_no_leak = X_valid.drop(columns=['repayment_history_score'])

# Modèle sans fuite
pipeline_no_leak = make_pipeline(RandomForestClassifier(n_estimators=100, random_state=42))
pipeline_no_leak.fit(X_train_no_leak, y_train)

# Évaluation
accuracy_no_leak = pipeline_no_leak.score(X_valid_no_leak, y_valid)
print(f"Accuracy sans fuite : {accuracy_no_leak:.2f}")
```

#### **5. Comparaison des résultats**
Comparez les scores de précision avec et sans fuite pour constater l'impact.

---

### Résultat attendu :
- **Avec fuite :** Le modèle affiche une précision élevée (ex. 0.98), car il utilise des informations directement liées à la cible.
- **Sans fuite :** La précision est plus réaliste (ex. 0.75), mais le modèle généralisera mieux sur de nouvelles données.

---

Si vous souhaitez exécuter ce code ou ajouter des étapes spécifiques (par exemple, utiliser des pipelines scikit-learn), je peux vous guider !